In [514]:
"""
@author: SergioSJS
"""
import numpy as np
from random import seed, randrange

In [607]:
dicX = {'o': 0.0, 'x' : 1.0, 'b' : -1.0}
dicY = {'negative' : -1, 'positive' : 1}

def replace(data, dic):
    ndata = np.zeros(data.shape)
    for r, row in enumerate(data):
        for c, col in enumerate(row):
            ndata[r, c] = dic.get(col)
    return ndata

def print_tic_tac_toe(tup):
    line = ''
    for l, it in enumerate(tup):
        line = line + ' ' + it
        if (l+1) % 3 == 0:
            print line
            line = ''
            
def kfold(data, folds=5):
    data_splited = list()
    data_copy = list(range(len(data)))
    fold_size = int(len(data)/folds)
    

       
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(data_copy))
            fold.append(data_copy.pop(index))
        data_splited.append(fold)
    
    for i in range(folds):
        test_list = list()
        train_list = list()
        train_temp = list()
        
        rangeF = range(folds)
        rangeF.pop(i)
        for i2 in rangeF:
            train_temp.append(data_splited[i2])
            
        train_list.append([y for x in train_temp for y in x])
        test_list.append(data_splited[i]) 
    
        yield train_list, test_list
    
            
#Load data
dataset = np.loadtxt('tic-tac-toe.data', delimiter=',', dtype=str)
xData = dataset[:,0:-1]
yData = dataset[:,-1:]

n_xData = replace(xData, dicX)
n_yData = replace(yData, dicY)

In [624]:
class DesicionStump():
    def __init__(self, xData, yData):
        self.xData = np.array(xData)
        self.yData = np.array(yData)     
        self.Qtd = self.xData.shape[1]
        
    def stump_train(self, W, steps=3, verbose=0):
        minErr = float('inf')
        _value = 0
        _att = 0
        _label = 0
        self.W = np.array(W)
        
        for att in range(self.Qtd):
            value, err = self._best_stump(att, steps, 1)
            if (err < minErr):
                minErr = err
                _value = value
                _att = att
                _label = 1
        for it in range(self.Qtd):
            value, err = self._best_stump(att, steps, -1)
            if (err < minErr):
                minErr = err
                _value = value
                _att = att
                _label = -1
        
        self.value = _value
        self.att = _att
        self.label = _label
        
        if verbose == 1:
            print 'att.:',_att,'- value:',_value,'- label:',_label, '- minErr:', minErr
        
        return minErr
            
    def _best_stump(self, att, steps, label):
        limite_up = np.max(self.xData[:, att])+1
        limite_bottom = np.min(self.xData[:, att])
        interval = (limite_up-limite_bottom)/steps
        
        _value = 0
        _minErr = float('inf')
        
        for value in np.arange(limite_bottom, limite_up, interval):
            # This code only considerate categorical data
            predict = self._predict_categorical_train(att, value, label)
            
            err = 0
            
            for i, d in enumerate(self.yData):
                err += (predict[i] != d) * self.W[i]
                
            if err < _minErr:
                _minErr = err
                _value = value
        return _value, _minErr
            
    def _predict_categorical_train(self, att, value, label):
        predict = np.zeros(self.yData.shape)
        for id, x in enumerate(self.xData[:,att]):
            if x == value:
                predict[id] = 1*label
            else:
                predict[id] = -1*label       
        return predict
    
    def predict_categorical(self, xTest):
        xTest=np.array(xTest)
        predict = np.zeros(xTest.shape[0])
        for id, x in enumerate(xTest[:,self.att]):
            if x == self.value:
                predict[id] = 1*self.label
            else:
                predict[id] = -1*self.label       
        return predict
            

In [627]:
class AdaBoost():
    def __init__(self, xData, yData):
        self.xData = np.array(xData)
        self.yData = np.array(yData)
        self.sums = np.zeros(self.yData.shape)
        # Initialize weights
        self.W = np.ones((self.xData.shape[0],1)).flatten()/self.xData.shape[0]

        self.dStump = DesicionStump
    
    def train(self, M=5, verbose=0):
        self.alpha = {}
        self.models = {}
        self.verbose = verbose
        
        for i in range(M):
            self.models[i] = self.dStump(self.xData, self.yData)
            
            e = self.models[i].stump_train(self.W, verbose = verbose)
            self.alpha[i] = (1/2.)*np.log((1-e)/e)
            res = self.models[i].predict_categorical(self.xData)
            
            Wz = self.W*np.exp(-self.alpha[i]*self.yData.flatten()*res.flatten())
            
            self.W = (Wz/Wz.sum()).flatten()
            
    def predict(self, testX):
        testX = np.array(testX)
        sums=np.zeros(testX.shape[0])
        
        for i in range(len(self.models)):
            sums = sums+self.models[i].predict_categorical(testX).flatten()*self.alpha[i]
               
        prev_y=np.zeros(np.array(sums).shape)
        prev_y[sums>=0]=1
        prev_y[sums<0]=-1
        
        return prev_y
        
           

In [634]:
seed(1)
for train, test in kfold(n_yData):
    ada = AdaBoost(n_xData[train], n_yData[train])
    ada.train(1000, verbose = 0)
    
    aa = ada.predict(n_xData[test])

    err = 0.
    acc = 0.
    for i, a in enumerate(n_yData[test]):
        if a == aa[i]:
            acc += 1
        else:
            err += 1
        #print i, a, n_yData[i]
    print "acc", acc/(acc+err)

acc 0.973821989529
acc 0.973821989529
acc 0.989528795812
acc 0.979057591623
acc 0.984293193717


100 = acc 0.746346555324
200 = acc 0.80375782881
500 = acc 0.903966597077
1000 = acc 0.978079331942
1500 = acc 0.983298538622